## Ejercicio III:

In [1]:
# Importando librerías

#!pip install dateparse
#!pip install openpyxl
import pandas as pd
import os
import datetime
import dateparser
import numpy as np

import warnings
warnings.simplefilter("ignore")

#### III.1 Creando el archivo df_sii.csv

In [2]:
df2019 = pd.read_csv("https://raw.githubusercontent.com/patoram123/tarea1-add/master/data/interim/d2019.csv",sep=",")
df2020 = pd.read_csv("https://raw.githubusercontent.com/patoram123/tarea1-add/master/data/interim/d2020.txt",sep=" ")

In [3]:
# Combinando ambos datasets
df_concat_0= pd.concat((df2019,df2020),axis=0)

# Reseteando indices para evitar errores
df_concat_0.reset_index(drop=True, inplace=True)

# Realizando melt segun instrucciones
df_sii = df_concat_0.melt(id_vars=['Día', 'year'], var_name='mes', value_name='valor')
df_sii.head()

,Día,year,mes,valor
0,1,2019.0,Ene,NaN
1,2,2019.0,Ene,694.77
2,3,2019.0,Ene,697.09
3,4,2019.0,Ene,697.64
4,5,2019.0,Ene,NaN


In [4]:
# Exportando como csv
df_sii.to_csv("..\data\interim\df_sii.csv",sep=",", index=False)

#### III.2 Usando el método groupby en df_sii:

- Al utilizar `as_index = False` las columnas utilizadas para agrupar se mantienen como columnas y no pasan a ser indices del Dataframe.


- Al agrupar solo por `mes` se calcula el valor por mes, combinando ambos años para el cálculo, además al año ser un número se calcula el promedio de ambos quedando un valor de 2019.5 para todos los meses. Mientras que utilizando `[year,mes]` los años se separan y se calcula el promedio de cada año por separado.

In [5]:
df_sii.groupby(by="mes", as_index = True).agg("mean")

,Día,year,valor
mes,,,
Abr,16.0,2019.5,760.389048
Ago,16.0,2019.5,749.182619
Dic,16.0,2019.5,752.561750
Ene,16.0,2019.5,724.854773
Feb,16.0,2019.5,726.342000
Jul,16.0,2019.5,735.394318
Jun,16.0,2019.5,744.299024
Mar,16.0,2019.5,755.523488
May,16.0,2019.5,753.659500


In [6]:
df_sii.groupby(by=["mes","year"], as_index = False).agg("mean")

,mes,year,Día,valor
0,Abr,2019.0,16.0,667.399048
1,Abr,2020.0,16.0,853.379048
2,Ago,2019.0,16.0,713.703333
3,Ago,2020.0,16.0,784.661905
4,Dic,2019.0,16.0,770.390500
5,Dic,2020.0,16.0,734.733000
6,Ene,2019.0,16.0,677.061818
7,Ene,2020.0,16.0,772.647727
8,Feb,2019.0,16.0,656.304500
9,Feb,2020.0,16.0,796.379500


#### III.3 Creando columna dia_de_la_semana

Al no haber operaciones bancarias en días no hábiles (feriados, Sábados y Domingos) esos días se registra un valor nulo (o NaN)

In [7]:
type(df_sii['mes'][1])

str

In [8]:
def get_dow(day, month, year):
    if type(month) == str:
        dt_str = str(int(day)) + ' ' + month + ' ' + str(int(year))
    else:
        dt_str = str(int(day)) + ' ' + str(int(month)) + ' ' + str(int(year))
    # Como existen días no válidos se debe hacer atrapar el error y generar una nueva etiqueta
    try:
        dow = dateparser.parse(dt_str, languages=['es']).weekday()
    except:
        dow = 7
    week = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo', 'Not valid']
    return week[dow]
    

In [9]:
df_sii['dia_de_la_semana'] = df_sii.apply(lambda x: get_dow(x['Día'], x['mes'], x['year']), axis=1)
display(df_sii.head())
display(df_sii.tail())

,Día,year,mes,valor,dia_de_la_semana
0,1,2019.0,Ene,NaN,Martes
1,2,2019.0,Ene,694.77,Miércoles
2,3,2019.0,Ene,697.09,Jueves
3,4,2019.0,Ene,697.64,Viernes
4,5,2019.0,Ene,NaN,Sábado


,Día,year,mes,valor,dia_de_la_semana
739,27,2020.0,Dic,NaN,Domingo
740,28,2020.0,Dic,710.26,Lunes
741,29,2020.0,Dic,710.64,Martes
742,30,2020.0,Dic,711.24,Miércoles
743,31,2020.0,Dic,NaN,Jueves


In [10]:
df_sii[df_sii["valor"].isnull()]

,Día,year,mes,valor,dia_de_la_semana
0,1,2019.0,Ene,NaN,Martes
4,5,2019.0,Ene,NaN,Sábado
5,6,2019.0,Ene,NaN,Domingo
11,12,2019.0,Ene,NaN,Sábado
12,13,2019.0,Ene,NaN,Domingo
...,...,...,...,...,...
732,20,2020.0,Dic,NaN,Domingo
737,25,2020.0,Dic,NaN,Viernes
738,26,2020.0,Dic,NaN,Sábado
739,27,2020.0,Dic,NaN,Domingo


#### III.4 Explorando el archivo dolar_observado_bc.xlsx

- Luego de realizar todas las operaciones para separar por año y hacer los Dataframes comparables, en los años 2019 y 2020, todos los valores coinciden. 

In [11]:
#creación de un df para cada hoja del archivo excel
dolar_observado_bc = pd.read_excel(
    "https://raw.githubusercontent.com/patoram123/tarea1-add/master/data/raw/dolar_observado_bc.xlsx",
                                 skiprows=2, dtype={'Periodo':str }, sheet_name=None)

df_1980 = dolar_observado_bc.get('80')
df_1990 = dolar_observado_bc.get('90')
df_2000 = dolar_observado_bc.get('00')
df_2010 = dolar_observado_bc.get('10')
df_2020 = dolar_observado_bc.get('20')

#creacion de un solo data frame
df_bc_all=pd.concat((df_1980,df_1990,df_2000,df_2010,df_2020),axis=0)

# Creacion de nuevas columnas
df_bc_all["Periodo"] = df_bc_all["Periodo"].str[0:10]
df_bc_all[["year", "mes", "dia"]] = df_bc_all["Periodo"].str.split("-", expand = True)

# Ordenando columnas indices
df_bc_all.columns = ["Periodo","Dolar","year","mes","dia"]
df_bc_all.reset_index(drop=True, inplace=True)

df_bc_all

,Periodo,Dolar,year,mes,dia
0,1982-08-09,55.65,1982,08,09
1,1982-08-10,62.79,1982,08,10
2,1982-08-11,64.07,1982,08,11
3,1982-08-12,60.52,1982,08,12
4,1982-08-13,56.87,1982,08,13
...,...,...,...,...,...
14628,2022-08-27,NaN,2022,08,27
14629,2022-08-28,NaN,2022,08,28
14630,2022-08-29,900.22,2022,08,29
14631,2022-08-30,890.90,2022,08,30


In [12]:
# Guardando archivos por año en nuevo directorio

path = "..\data\processed\dolar_bc_year"
try: 
    os.mkdir(path) 
except OSError as error: 
    print(error) 

# Creación de csv agrupando por "year"
for (year), group in df_bc_all.groupby(['year']):
    group.to_csv(path + f'\dolar_bc_{str(year)}.csv', index=False)

[Errno 17] File exists: '..\\data\\processed\\dolar_bc_year'


In [13]:
#importar csv de los años 2019 y 2020
d_2019_2=pd.read_csv(path + '\dolar_bc_2019.csv')
d_2020_2=pd.read_csv(path + '\dolar_bc_2020.csv')

df_bc_2019_2020 = pd.concat((d_2019_2,d_2020_2), axis=0)
df_bc_2019_2020.reset_index(drop=True, inplace = True)

df_bc_2019_2020

,Periodo,Dolar,year,mes,dia
0,2019-01-01,NaN,2019,1,1
1,2019-01-02,694.77,2019,1,2
2,2019-01-03,697.09,2019,1,3
3,2019-01-04,697.64,2019,1,4
4,2019-01-05,NaN,2019,1,5
...,...,...,...,...,...
726,2020-12-27,NaN,2020,12,27
727,2020-12-28,710.26,2020,12,28
728,2020-12-29,710.64,2020,12,29
729,2020-12-30,711.24,2020,12,30


In [14]:
# Arreglando Dataframes para hacerlos comparables

df_sii_comp = df_sii[df_sii['dia_de_la_semana'] != 'Not valid'].sort_values(by=["Día", "year", "valor"])
df_sii_comp.reset_index(drop=True, inplace=True)

df_bc_2019_2020_comp = df_bc_2019_2020.sort_values(by=["dia", "year", "Dolar"])
df_bc_2019_2020_comp.reset_index(drop=True, inplace=True)

# Comparando ambos DFs
df_sii_comp[(df_sii_comp["valor"] != df_bc_2019_2020_comp["Dolar"]) & ~df_sii_comp["valor"].isnull()]

,Día,year,mes,valor,dia_de_la_semana


#### III.5 Contando NaNs

- Al revisar los días con más NaNs podemos notar que se dan en días festivos, al ser estos fijos en todos los años era esperable encontrar este resultado (18 y 19 de Septiembre, Navidad, Año Nuevo, etc.). Esto permite inferir (o confirmar) que el dolar no cotiza en días no hábiles.


- Sábados y Domingos rotan más en función de la combinación "día/mes", por lo que no es tan extraño encontrar counts más bajos y una mayor cardinalidad, esto considerando que los festivos son en "día/mes" fijos en muchos casos.


In [15]:
df_sii_comp.columns = ['dia', 'year', 'mes', 'valor', 'dia_de_la_semana']
df_sii_comp.head()

,dia,year,mes,valor,dia_de_la_semana
0,1,2019.0,Mar,651.79,Viernes
1,1,2019.0,Feb,657.81,Viernes
2,1,2019.0,Abr,678.53,Lunes
3,1,2019.0,Jul,679.15,Lunes
4,1,2019.0,Ago,700.82,Jueves


In [ ]:
# Considerando que la transformación de las fechas a días de semana se puede
# realizar utilizando la función get_dow se utilizará eso para df_bc_all

# Sería necesario buscar como extender y hacer merge con otra variable, pero esta solución es más simple

df_bc_all['dia_de_la_semana'] = df_bc_all.apply(lambda x: get_dow(x['dia'], x['mes'], x['year']), axis=1)

display(df_bc_all.head())
display(df_bc_all.tail())


In [ ]:
# Creando variable que combina día y mes
df_bc_all["dia/mes"] = df_bc_all["dia"].astype(str) + "/" + df_bc_all["mes"].astype(str)

In [ ]:
# Separando dias de semana y fin de semana

df_bc_all_semana = df_bc_all[~df_bc_all["dia_de_la_semana"].isin(["Sábado", "Domingo"])]
df_bc_all_fds = df_bc_all[df_bc_all["dia_de_la_semana"].isin(["Sábado", "Domingo"])]

# Contando NaNs en Días de semana
# Eligiendo Periodo de manera arbitraria para obtener el count, podría haber sido cualquier columnas distinta de Dolar

# Length 78
display(df_bc_all_semana[df_bc_all_semana["Dolar"].isnull()]\
                .groupby("dia/mes").count()["Periodo"]\
                .sort_values(ascending=False)\
                .head(15))

# Length 366
display(df_bc_all_fds[df_bc_all_fds["Dolar"].isnull()]\
                .groupby("dia/mes").count()["Periodo"]\
                .sort_values(ascending=False)\
                .head(15))

#### III.6 Calculando promedios en Días de cumpleaños y comparando

- Considerando que la muestra de los días de cumpleaños es bastante pequeña esperaba ver mayores diferencias con el promedio para todos los días. En general, las diferencias son pequeñas, esto podría significar que el precio del dolar varía poco en un año. Sin embargo, algunos años poseen diferencias más significativas que otros, esto podría estar algo relacionado a años complicados para la economía nacional (poco crecimiento en 2001 o crisis del 2008 por ejemplo)

In [ ]:
#creación de un df para cada hoja del archivo excel
cols = ["n", "Nombre", "Apellido", "dia/mes"]
df_integrantes = pd.read_csv(
    "https://raw.githubusercontent.com/patoram123/tarea1-add/master/Integrantes-patoram123.md"
            ,sep=" "
            ,header=None 
            ,names=cols)

df_integrantes

In [ ]:
# Promedio en los días de cumpleaños
prom_cumple = df_bc_all[df_bc_all["dia/mes"].isin(df_integrantes["dia/mes"])]\
            .groupby("year").mean()

prom_cumple.columns = ["Dolar_cumple"]

# Promedio por año considerando todos los días
prom_all = df_bc_all.groupby("year").mean()
prom_all.columns = ["Dolar_all"]

proms = pd.concat([prom_cumple, prom_all], axis=1)

proms["diff"] = proms["Dolar_cumple"] - proms["Dolar_all"]

proms